# DML e Técnicas Avançadas de Consultas 🧠

![SQL girl meme](https://live.staticflickr.com/65535/49198404911_55085b0307_z.jpg)

## Revisão Aula 02

> Funções: `GROUP BY`, `UNION / UNION ALL`

![SQL Join Image](../img/aula02/join_types0.jpg "SQL Join image")

> Técnicas: Subqueries, Tabelas temporárias - locais, globais e *Common Table Expressions* (CTEs)

**********

![Grupos SQL](../img/aula01/grupos_sql.png)

Os tipos da linguagem SQL são:

- **DQL - Data Query Language** - Linguagem de Consulta de dados.
    - São os comandos de consulta. Exemplos: `SELECT`, `FROM`
- **DML - Data Manipulation Language** - Linguagem de Manipulação de Dados.
    - São os comandos que interagem com os dados dentro das tabelas. Exemplos: `INSERT`, `DELETE`, e `UPDATE`.
- DTL - Data Transaction Language - Linguagem de Transação de Dados.
    - São os comandos para controle de transação. Exemplos: `BEGIN TRANSACTION`, `COMMIT` e `ROLLBACK`

## Inserindo novos dados nas tabelas

Para inserir novos registros em uma tabela existente, o comando `INSERT INTO` deve ser utilizado. A sintaxe básica é a seguinte:

```sql
INSERT INTO <table_name> (coluna1, coluna2, coluna3, ...)
VALUES (valor1, valor2, valor3, ...);
```

Devemos ter atenção às restrições existentes no banco de dados, com relação à integridade dos registros.

In [ ]:
-- Exemplo inserir registro único
INSERT INTO station 
(
    [ID],
    [Name],
    [zmine],
    [zdescription],
    [Depth],
    [stn_type]
)
VALUES (
    -- 1,           -- Erro PK (ID 1 já existe)
    1500,
    'TESTE-01',
    'Treinamento SQL',
    'Teste INSERT Treinamento SQL',
    150,
    'Teste'
)

-- Exibir registro recém inserido
SELECT * FROM station WHERE [Name] = 'TESTE-01'

In [ ]:
-- Exemplo inserir múltiplos registros
INSERT INTO lithology ([Station], [from_], [to_], [soil_type], [comment])
VALUES
    (1500, 0, 5, 'A', 'Teste'),
    (1500, 5, 15, 'B', 'Teste'),
    (1500, 15, 22, 'C', 'Teste'),
    (1500, 22, 30, 'D', 'Teste')

-- Exibir registros cadastrados
SELECT * FROM lithology WHERE Station = 1500

In [ ]:
-- Melhorar formatação
SELECT
    s.Name          AS [Codigo HGA]
    ,s.stn_type     AS [Tipo Ponto]
    ,l.from_        AS [De (m)]
    ,l.to_          AS [Ate (m)]
    ,l.soil_type    AS [Litologia]
FROM lithology l
    JOIN station s
        ON l.Station = s.ID
WHERE s.[Name] = 'TESTE-01'

In [ ]:
-- Possíveis erros...

INSERT INTO lithology ([Station], [from_], [to_], [soil_type], [comment])
VALUES
    -- (1501, 0, 1, 'Violação FK', 'Errrou')   -- Erro FK
    -- ('Ponto', 0, 1, 'Tipo dado incorreto', 'Não passarás')   -- Erro tipo de dado

    -- (1500, 0, 10, 'A', 'Ok...'),   -- Estrutura Ok, logicamente inconsistente.
    -- (1500, 5, 12, 'B', 'Ok...')
    
    -- (1500, 0, 10, 'Violação PK', 'Duplicidade')

> #### 💡 Dica: Para a importação de dados em .xlsx ou .csv, podemos utilizar módulos auxiliares como o **Import Wizard** do SSMS.

##### Exercício: Importação de dados de medição de campo

## Deletando registros

Para deletar registros de uma tabela, utiliza-se a instrução `DELETE FROM <tabela>`. 

Para adicionar uma condição de remoção, utilizamos a sintaxe:
```sql
DELETE FROM <tabela> WHERE <condition>;
```

> 💡 Dica: 
> - Para remover **intencionalmente** todas as linhas de uma tabela, o comando `TRUNCATE TABLE <tabela>` é mais eficiente.
> - Ainda, existe o comando `DROP TABLE <tabela>`, que remove a tabela totalmente, inclusive índices, *triggers*, restrições e permissões.

📚 Recomendação de leitura:

- [DELETE *vs* TRUNCATE *vs* DROP](https://sergioleitaodba.wordpress.com/2014/10/20/sql-diferenca-entre-os-comandos-truncate-delete-e-drop/)

***

In [ ]:
-- Removendo registros

DELETE FROM lithology WHERE [Station] = 1500 AND [comment] = 'Ok...'

SELECT * FROM lithology WHERE Station = 1500

## Atualizando registros de uma tabela

Para atualizar registros, utilizamos o operador `UPDATE`. A sintaxe segue o seguinte formato:

```sql
UPDATE <tabela>
SET
    coluna1 = valor1,
    [coluna2 = valor2,]
    [coluna3 = valor3,]
WHERE
    <condição>;
```

In [ ]:
-- Exemplo UPDATE básico
UPDATE lithology 
    SET [to_] = 100 WHERE Station = 1500 AND [to_] = 30

SELECT * FROM lithology WHERE Station = 1500

## Muito cuidado ⚠️ 
### UPDATE/DELETE sem WHERE!

![UPDATE SEM WHERE](../img/aula03/update_sem_where.jpg)

![UPDATE SEM WHERE](../img/aula03/update_sem_where_cert.png)

> “*Quando executarem algum dos comandos UPDATE ou DELETE sem WHERE pensem que está matando alguém, um gato, um cachorro. É um dos piores cenários que existe. Muito cuidado!*”

In [ ]:
-- -- Substituir as litologias do ponto recém criado..
-- UPDATE lithology SET soil_type = 'Litologia' WHERE Station = 1500

-- Erro gravíssimo!
UPDATE lithology SET soil_type = 'Litologia' --WHERE Station = 1500

SELECT * FROM lithology

### Evitando problemas ... DTL (Data Transaction Language)

In [ ]:
BEGIN TRAN

UPDATE lithology SET comment = 'Comentario' WHERE Station = 1500

ROLLBACK TRAN;
--COMMIT TRAN;
GO

SELECT * FROM lithology WHERE Station = 1500

# Window Functions

## Ranking de grupos de variáveis por um contador 🎾

Por vezes, é necessário **classificar os campos por grupos**, em alguma ordem, para que possa executar análises. Estas situações poderão ser:

1. Quando os dados anuais estão a ser atualizados pelo menos duas vezes por ano e pretende identificar quais as linhas dos seus dados mais recentes com base numa coluna que identifica quando os dados foram importados.
2. Quando você quiser remover linhas 'duplicadas' em sua tabela com base em uma coluna ordenada.

Em outros termos, podemos pensar nisso como **ordenar** a tabela, mas em vez de ordenar todos os dados da tabela, estamos classificando/ordenando grupos e subgrupos dentro da tabela.

Para criar ***rankings*** por um contador, devemos fazer o seguinte:

1. Criar uma nova coluna, `[Nova Coluna]`
2. Atribuir o seguinte valor:

`[Nova Coluna] = [ROW_NUMBER() OVER (PARTITION BY [coluna_1], ... [coluna_n]) ORDER BY [coluna_1], ... [coluna_n] ASC] ASC/DESC;`

### Exemplo: Retornar TOP 3 pontos com mais amostras hidroquímicas por complexo

In [ ]:
WITH n_amostras AS (
	SELECT
		s.zcomplex				AS [Complexo]
		,s.[Name]				AS [Codigo HGA]
		,COUNT(a.sample_id)		AS [N Amostras]
	FROM parameter_sample a
		JOIN station s
			ON a.Station = s.ID
	GROUP BY s.zcomplex, s.Name
),

top_3 AS (
	SELECT
		n_amostras.*
		,ROW_NUMBER() OVER(PARTITION BY n_amostras.Complexo ORDER BY n_amostras.[N Amostras] DESC)	AS rn
	FROM n_amostras 
)

SELECT *
FROM top_3
WHERE rn <= 3

In [ ]:
-- Função RANK()

SELECT
    s.zcomplex				AS [Complexo]
    ,s.[Name]				AS [Codigo HGA]
    ,COUNT(a.sample_id)		AS [N Amostras]
    ,RANK() OVER(PARTITION BY s.zcomplex ORDER BY COUNT(a.sample_id) DESC) AS [Ranking]
FROM parameter_sample a
    JOIN station s
        ON a.Station = s.ID
GROUP BY s.zcomplex, s.Name

In [ ]:
-- Outras funções de RANKING

WITH rank AS (
    SELECT
        s.zcomplex				AS [Complexo]
        ,s.[Name]				AS [Codigo HGA]
        ,COUNT(a.sample_id)		AS [N Amostras]
        ,RANK() OVER(PARTITION BY s.zcomplex ORDER BY COUNT(a.sample_id) DESC) AS [Ranking]
        -- ,DENSE_RANK() OVER(PARTITION BY s.zcomplex ORDER BY COUNT(a.sample_id) DESC) AS [Dense_Ranking]
        -- ,NTILE(10) OVER(PARTITION BY s.zcomplex ORDER BY COUNT(a.sample_id) DESC) AS [Quartil]
    FROM parameter_sample a
        JOIN station s
            ON a.Station = s.ID
    GROUP BY s.zcomplex, s.Name
)

SELECT * FROM rank WHERE Ranking <= 3

### Window + Agregação 

In [ ]:
SELECT DISTINCT
    s.Name
    ,s.zmatrix
    ,s.zcomplex
    ,s.stn_type
    ,COUNT(a.sample_id) OVER(PARTITION BY s.Name)       AS [Qtde Ponto]
    ,COUNT(a.sample_id) OVER(PARTITION BY s.stn_type)   AS [Qtde Tipo]
    ,COUNT(a.sample_id) OVER(PARTITION BY s.zcomplex)   AS [Qtde Complexo]
FROM station s
    JOIN parameter_sample a
        ON s.ID = a.Station

In [ ]:
WITH groups AS (
    SELECT DISTINCT
        s.Name
        ,s.zmatrix
        ,s.zcomplex
        ,s.stn_type
        ,COUNT(a.sample_id) OVER(PARTITION BY s.Name)       AS [Qtde Ponto]
        ,COUNT(a.sample_id) OVER(PARTITION BY s.stn_type)   AS [Qtde Tipo]
        ,COUNT(a.sample_id) OVER(PARTITION BY s.zcomplex)   AS [Qtde Complexo]
    FROM station s
        JOIN parameter_sample a
            ON s.ID = a.Station
)

SELECT
    groups.*
    ,[Representatividade %] = [Qtde Ponto] / [Qtde Tipo]
    -- ,[Representatividade %] = CAST([Qtde Ponto] AS FLOAT) / CAST([Qtde Tipo] AS FLOAT)
FROM groups

📚 Recomendações de leitura:

- [SQL: Window Function I – Revisão dos Conceitos.](https://managebi.com/2022/09/30/sql-window-function-conceitos)
- [Ranking Functions (T-SQL)](https://learn.microsoft.com/en-us/sql/t-sql/functions/ranking-functions-transact-sql?view=sql-server-ver16)
- [SQL Window Functions](https://mode.com/sql-tutorial/sql-window-functions/)
- [How to Use Window Functions in SQL – with Example Queries](https://www.freecodecamp.org/news/window-functions-in-sql/)

# Pivotando dados - PIVOT 💃

Operações de "**pivotagem**" para transpor linhas para colunas, são úteis em vários cenários, como o *one-hot encoding* de dados categóricos antes de aplicar modelos de aprendizado de máquina em Python ou simplesmente para melhor formatação.

Uma boa forma de pensar sobre **pivotagem** é a utilização de uma coluna (**pivô**) para rearranjar a visualização da tabela de um formato mais compacto para um formato mais amplo (menos para mais colunas). 

A clássica ferramenta de **tabela dinâmica (*pivot table*)** do Excel é um ótimo exemplo.

![tDin](../img/aula03/pivot_table.png)

Quando vamos **pivotar** os dados, é útil saber quais os valores únicos da coluna que está sendo pivotada. No SQL, utilizamos a função `PIVOT` para realizar esta operação.

### Exemplo: Pluviometria acumulada mensal para o ponto ID = 404

In [ ]:
SELECT 
    Station
    ,MONTH([date_])                                             AS [Mês]
    ,IIF(MONTH([date_]) BETWEEN 4 AND 9, 'Seco', 'Chuvoso' )    AS [Período]
    ,YEAR([date_])                                              AS [Ano]
    ,SUM([total_precipitation])                                 AS [Precipitacao]
FROM [meteorology]
WHERE Station = 404
GROUP BY Station, YEAR([date_]), MONTH([date_])
ORDER BY Station, YEAR([date_]), MONTH([date_])

In [ ]:
WITH chuva AS (
      SELECT 
            Station
            -- ,MONTH([date_])                                             AS [Mês]
            ,IIF(MONTH([date_]) BETWEEN 4 AND 9, 'Seco', 'Chuvoso')     AS [Período]
            ,YEAR([date_])                                              AS [Ano]
            ,SUM([total_precipitation])                                 AS [Precipitacao]
      FROM [meteorology]
      WHERE Station = 404
      GROUP BY Station, YEAR([date_]), MONTH([date_])
)

-- -- Verificar valores únicos na coluna a ser pivotada
-- SELECT DISTINCT [Período] FROM [precipitacao]

-- Pivotar coluna [Período] somando as precipitacoes
SELECT Station, Ano, [Seco], [Chuvoso], [Seco] + [Chuvoso] AS [Total]
FROM [chuva]
PIVOT
(
	SUM([Precipitacao]) 
	FOR [Período] IN ([Seco], [Chuvoso])
) AS pivot_table;

#  Pivotando dados - UNPIVOT 💃

**Unpivoting** data so that you have columns being transposed to rows is useful when trying to structure/format your data consistently for different analytical software programs or for presentation purposes.

A good way to think about **unpivoting** is that you are going from your table being wider and shorter to it being thinner and longer.

To **unpivot** your data, it is quite involved to describe at a high-level so the example below will provide a better description of how it works.

A operação inversa da **pivotagem** ("despivotagem" ou "*unpivot*"), descrita anteriormente, se dá pela transposição de colunas para linhas (mais para menos colunas). Esta é uma operação útil para estruturar/formatar dados no formato tabular (normalmente trabalhado em bancos de dados ou sistemas em geral).

No SQL, usamos a função `UNPIVOT` para executar esta operação.

### Exemplo: Convertendo tabela _"field_measurements"_ para o formato tabular.

In [ ]:
SELECT TOP 10 * FROM field_measurements
SELECT TOP 10 * FROM parameter_result   

In [ ]:
-- Aplicar operação inversa da pivotagem na tabela field_measurements

SELECT
	Station
	,[fm_date]				AS [Data Amostra]
	,measurement_method		AS [Metodo Amostra]
	,[Parametro]
	,[Resultado Original]
FROM field_measurements
UNPIVOT
(
	[Resultado Original]
	FOR [Parametro] IN ([ph], [temperature], [conductivity], [zorp], [total_dissolved_solids], [dissolved_oxygen], [zresistivity], [zturbidity], [zpressure])
) AS tabular;

In [ ]:
-- Renomear os parâmetros

WITH field_measurements_renamed AS (
    SELECT
        Station
        ,[fm_date]                      AS [Data Amostra]
        ,[ph]                           AS [pH]
        ,[temperature]                  AS [Temperatura]
        ,[conductivity]                 AS [Condutividade Elétrica]
        ,[zorp]                         AS [ORP]
        ,[total_dissolved_solids]       AS [Solidos Totais Dissolvidos]
        ,[dissolved_oxygen]             AS [Oxigênio Dissolvido]
        ,[zresistivity]                 AS [Resistividade]
        ,[zturbidity]                   AS [Turbidez]
        ,[zpressure]                    AS [Pressão]
        ,[measurement_method]           AS [Metodo Amostra]
    FROM field_measurements f
)

SELECT
	Station
	,[Data Amostra]
    ,[Metodo Amostra]
	,[Parametro]
	,[Resultado Original]
FROM field_measurements_renamed
UNPIVOT
(
	[Resultado Original]
	FOR [Parametro] IN (
        [pH], [Temperatura], [Condutividade Elétrica], [ORP], [Solidos Totais Dissolvidos], [Oxigênio Dissolvido], [Resistividade], [Turbidez], [Pressão]
    )
) AS tabular;